In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph,START,END


In [5]:
class ChatState(TypedDict):
    user_input:str
    response: str

In [ ]:
from langgraph.store.memory import InMemoryStore
store=InMemoryStore()

def memory_namespace(user_id:str):
    return (user_id,"profile")

In [7]:
def load_memory(state:ChatState,config):
    user_id=config["configurable"]["user_id"]
    memories=store.search(memory_namespace(user_id))

    if memories:
        last_memory=memories[-1].value
    else:
        last_memory={}
    
    return {
        "response":f"Loaded memory: {last_memory}"
    }

In [8]:
def chat_node(state:ChatState):
    user_input=state['user_input']

    if 'pizza' in user_input.lower():
        return {
            "response":"Nice I'll remember that"
        }
    return{
        "response":"Got it!"
    }

In [9]:
import uuid

def save_memory(state:ChatState,config):
    user_id=config['configurable']['user_id']
    text=state["user_input"]

    if "pizza" in text.lower():
        store.put(
            memory_namespace(user_id),
            str(uuid.uuid4()),
            {"food_preference":"pizza"}
        )
    return {}

In [11]:
workflow=StateGraph(ChatState)

workflow.add_node("load_memory",load_memory)
workflow.add_node("chat",chat_node)
workflow.add_node("save_memory",save_memory)

workflow.add_edge(START,"load_memory")
workflow.add_edge("load_memory","chat")
workflow.add_edge("chat","save_memory")
workflow.add_edge("save_memory",END)


from langgraph.checkpoint.memory import InMemorySaver
checkpointer=InMemorySaver()
graph=workflow.compile(checkpointer=checkpointer)

In [13]:
config={
    "configurable":{
        "thread_id":"thread-1",
        "user_id":"1"
    }
}

graph.invoke(
    {"user_input":"I like pizza","response":""},
    config
)

{'user_input': 'I like pizza', 'response': "Nice I'll remember that"}

In [14]:
config = {
    "configurable": {
        "thread_id": "thread-2",
        "user_id": "1"
    }
}

graph.invoke(
    {"user_input": "Hello", "response": ""},
    config=config
)


{'user_input': 'Hello', 'response': 'Got it!'}